# Install the relevant libraries

In [ ]:
!pip install transformers wikipedia newspaper3k GoogleNews pyvis -qq

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import wikipedia
from newspaper import Article, ArticleException
from GoogleNews import GoogleNews
import IPython
from pyvis.network import Network

# Load the REBEL model

In [3]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

# Load Sentiment Model

In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="TrajanovRisto/bert-esg")

In [9]:
def get_sentiment(text):
    sentence_class = {}
    for id, sent in enumerate(text.split('. ')):
      if len(sent) > 512:
        sent = sent[:512]
      sentence_class[id] = classifier(sent)
    return sentence_class
    

# Model output to realtions

In [5]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

# Split spans: from long text to KB

In [90]:
class KB():
    def __init__(self):
        self.relations = []
        self.entities = {}

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def add_entity(self, e):
        self.entities[e] = {'url': ''}

    def add_relation(self, r):
        entities = [r["head"], r["tail"]]

        for e in entities:
            self.add_entity(e)
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [91]:
def from_text_to_kb(text, span_length=128, verbose=False, num_return_sequences=3):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": num_return_sequences,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)


    id_sentence = {id:sent for id, sent in enumerate(text.split('. '))}
    # sentence_entities, entity_ner = get_ner(text)
    sentence_class = get_sentiment(text)
    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            for id, sent in id_sentence.items():
              if relation['head'] in sent and relation['tail'] in sent:
                relation['sentiment'] = sentence_class[id][0]['label']
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

# Filter and normalize entities with Wikipedia

- remove all entities that doesn't have a page on Wikipedia
- merge entities if they have the same wikipedia page

In [93]:
text = """
As evidence of the severe impacts from climate change mounts, policy makers, companies, and financial bodies are increasingly focused on the economic impacts1 from driving greenhouse gas (GHG) emissions to well-below 2 degrees Celsius below pre-industrial levels (including 1.5° C ambitions), as outlined in the Paris Agreement.
This focus has led many Chevron peers (including BP, Eni, Equinor, Repsol, Royal Dutch Shell, and Total) to commit to major GHG reductions, including setting “net zero emission” goals by 2050.2,3
Investors are also calling for high-emitting companies to test their financial assumptions and resiliency against substantial reduced-demand climate scenarios,4 and to provide investors insights about the potential impact on their financial statements.5,6,7
As of December 2020, Chevron Corporation had neither committed to net-zero emissions by 2050 across its value chain, nor disclosed how its financial assumptions would change from doing so.
In contrast, the audit reports for other high GHG-emitting companies clearly discussed this connection:
BP: how climate change and a global energy transition impacted the capitalization of exploration and appraisal costs and risks that oil and gas price assumptions could lead to financial misstatements;
Shell: how long-term price assumptions impacted by climate change could affect asset values and impairment estimates;
National Grid: noted estimates inconsistent with 2050 “net zero” commitments.
Additionally, in 2020, BP, Shell and Total reviewed their 2019 financial accounting practices in light of the accelerating low-carbon energy transition. All three subsequently adjusted critical accounting assumptions, resulting in material impairments, and disclosed how climate change affected the adjustments.
In October 2020, the International Energy Agency (IEA) issued a new “Net Zero 2050” scenario which describes what it would mean for the energy sector globally to reach net-zero GHG emissions by 2050.
This more aggressive global action to curtail climate change is consistent with a 1.5°C temperature increase globally.8
RESOLVED: Shareholders request that Chevron’s Board of Directors issue an audited report to shareholders on whether and how a significant reduction in fossil fuel demand, envisioned in the IEA Net Zero 2050 scenario, would affect its financial position and underlying assumptions. The Board should summarize its findings to shareholders by January 31, 2022, and the report should be completed at reasonable cost and omitting proprietary information.
Proponents recommend that in issuing the report, the company take account of information on:
Assumptions, costs, estimates, and valuations that may be materially impacted; and
The potential for widespread adoption of net-zero goals by governments and peers.9
Proponents recommend that the report be supported by reasonable assurance from an independent auditor.
"""

kb_proposal = from_text_to_kb(text)

In [94]:
kb_proposal.print()

Relations:
  {'head': 'severe impacts from climate change', 'type': 'has effect', 'tail': 'economic impacts1', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[0, 128]]}}
  {'head': 'economic impacts1', 'type': 'has cause', 'tail': 'severe impacts from climate change', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[0, 128]]}}
  {'head': 'severe impacts from climate change', 'type': 'facet of', 'tail': 'Paris Agreement', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[0, 128]]}}
  {'head': 'BP', 'type': 'significant event', 'tail': 'global energy transition', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[112, 240]]}}
  {'head': 'reduced-demand', 'type': 'facet of', 'tail': 'climate change', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[112, 240]]}}
  {'head': 'BP', 'type': 'participant in', 'tail': 'global energy transition', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[112, 240]]}}
  {'head': 'accelerating low-car

In [95]:
save_network_html(kb_proposal, filename='chevron_proposal.html')
IPython.display.HTML(filename=filename)

chevron_proposal.html


In [96]:
text = """
Chevron believes the future of energy is lower carbon and aims to equip stockholders with data and facts so that they can make informed choices as the world moves toward the global net-zero ambitions of the Paris agreement. In March 2021, the Company published its third Climate Change Resilience Report aligned with the Task Force for Climate-related Financial Disclosures. The report outlines Chevron’s approach to the energy transition, including testing the resiliency of Chevron’s portfolio against the International Energy Agency’s Sustainable Development Scenario (“SDS”) and Net Zero Scenario.
At its Investor Day on March 9, 2021, Chevron outlined the steps it is taking, and the challenges it faces, in working toward a net-zero future. This includes changes in portfolio, execution of projects identified through marginal abatement cost curves, and the policy, innovations, and offsets necessary to achieve net-zero.
Chevron uses long-term energy demand scenarios and a range of commodity prices to test its portfolio, guide investment strategies, and evaluate business risks to ensure it can deliver results under a range of potential futures. Chevron analyzes how various factors may combine to create alternative scenarios to stress-test its portfolio and integrate learnings into its decision making to remain competitive and resilient under multiple scenarios.
For longer-term scenarios, Chevron routinely uses external views to both inform and challenge its internal analysis. Chevron’s analysis includes scenarios forecasting emissions pathways that keep global warming to well below 2o C above pre-industrial levels, as well as scenarios forecasting net-zero emissions by 2050.1 One example of a lower carbon scenario tested against Chevron’s portfolio is the SDS. The SDS outlines one potential path to 2040 that achieves the objectives of recent clean energy policies, including the Paris Agreement, keeping global average temperatures well below 2o C above pre-industrial levels and putting the world on track to achieve net-zero emissions by 2070.
In 2020, more than 60 percent of Chevron’s total Scope 1 and Scope 2 equity GHG emissions were in regions with existing or developing carbon pricing policies.2 Chevron uses carbon prices and derived carbon costs in business planning, investment decisions, impairment reviews, reserves calculations, and assessment of carbon reduction opportunities. We believe that the Company’s portfolio is resilient, and that its governance structure, risk management, strategy, actions, and asset mix enable it to be flexible in response to potential changes in supply and demand, even in lower carbon scenarios.
The Company leverages its capabilities, assets, and expertise to focus on these three action areas that aim to deliver measurable progress: (1) lower carbon intensity cost efficiently, (2) increase renewables and offsets in support of our own business, and (3) invest in low-carbon technologies to enable commercial solutions.
Chevron’s employees are proud of their role in providing affordable, reliable, and ever-cleaner energy that people around the world depend on every day.
Given Chevron’s current disclosures and robust strategy, planning, and risk management processes, your Board believes that the requested report is unnecessary.
Therefore, your Board recommends that you vote AGAINST this proposal.
"""

kb_response = from_text_to_kb(text)
kb_response.print()

Relations:
  {'head': 'Sustainable Development Scenario', 'type': 'publisher', 'tail': 'International Energy Agency', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Sustainable Development Scenario', 'type': 'creator', 'tail': 'International Energy Agency', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Sustainable Development Scenario', 'type': 'author', 'tail': 'International Energy Agency', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Sustainable Development Scenario', 'type': 'facet of', 'tail': 'net-zero future', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[106, 234]]}}
  {'head': 'Sustainable Development Scenario', 'type': 'facet of', 'tail': 'net-zero', 'sentiment': 'Environmental Positive', 'meta': {'spans': [[106, 234]]}}
  {'head': 'Sustainable Development Scenario', 'type': 'part of', 'tail': 'net-zero future', 'sentiment': 'Environmental Positive', 'meta'

In [97]:
save_network_html(kb_response, filename='chevron_response.html')
IPython.display.HTML(filename=filename)

chevron_response.html


# Extract KB from web article

In [31]:
def get_sentiment(text):
    sentence_class = {}
    for id, sent in enumerate(text.split('. ')):
      if len(sent) > 512:
        sent = sent[:512]
      sentence_class[id] = classifier(sent)
    return sentence_class
    

In [56]:
def from_text_to_kb(text, article_url, span_length=128, article_title=None,
                    article_publish_date=None, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }
    id_sentence = {id:sent for id, sent in enumerate(text.split('. '))}
    # sentence_entities, entity_ner = get_ner(text)
    sentence_class = get_sentiment(text)
    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            for id, sent in id_sentence.items():
              if relation['head'] in sent and relation['tail'] in sent:
                relation['sentiment'] = sentence_class[id][0]['label']
            relation["meta"] = {
                article_url: {
                    "spans": [spans_boundaries[current_span_index]]
                }
            }
            kb.add_relation(relation, article_title, article_publish_date)
        i += 1

    return kb

In [78]:
class KB():
    def __init__(self):
        self.entities = {} # { entity_title: {...} }
        self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
          # meta: { article_url: { spans: [...] } } ]
        self.sources = {} # { article_url: {...} }

    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            article_url = list(r["meta"].keys())[0]
            source_data = kb2.sources[article_url]
            self.add_relation(r, source_data["article_title"],
                              source_data["article_publish_date"])

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r2):
        r1 = [r for r in self.relations
              if self.are_relations_equal(r2, r)][0]

        # if different article
        article_url = list(r2["meta"].keys())[0]
        if article_url not in r1["meta"]:
            r1["meta"][article_url] = r2["meta"][article_url]

        # if existing article
        else:
            spans_to_add = [span for span in r2["meta"][article_url]["spans"]
                            if span not in r1["meta"][article_url]["spans"]]
            r1["meta"][article_url]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e] = {'url': ''}

    def add_relation(self, r, article_title, article_publish_date):
        # manage new entities
        entities = [r["head"], r["tail"]]

        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        # r["head"] = entities[0]["title"]
        # r["tail"] = entities[1]["title"]
        # add source if not in kb
        article_url = list(r["meta"].keys())[0]
        if article_url not in self.sources:
            self.sources[article_url] = {
                "article_title": article_title,
                "article_publish_date": article_publish_date
            }
        
        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")
        print("Sources:")
        for s in self.sources.items():
            print(f"  {s}")

In [58]:
def get_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article

def from_url_to_kb(url):
    article = get_article(url)
    config = {
        "article_title": article.title,
        "article_publish_date": article.publish_date
    }
    kb = from_text_to_kb(article.text, article.url, **config)
    return kb

# Google News: extract KB from multiple articles

In [59]:
def get_news_links(query, lang="en", region="US", pages=1, max_links=100000):
    googlenews = GoogleNews(lang=lang, region=region)
    googlenews.search(query)
    all_urls = []
    for page in range(pages):
        googlenews.get_page(page)
        all_urls += googlenews.get_links()
    return list(set(all_urls))[:max_links]

def from_urls_to_kb(urls, verbose=False):
    kb = KB()
    if verbose:
        print(f"{len(urls)} links to visit")
    for url in urls:
        if verbose:
            print(f"Visiting {url}...")
        try:
            kb_url = from_url_to_kb(url)
            kb.merge_with_kb(kb_url)
        except ArticleException:
            if verbose:
                print(f"  Couldn't download article at url {url}")
    return kb

In [60]:
import pickle

def save_kb(kb, filename):
    with open(filename, "wb") as f:
        pickle.dump(kb, f)

def load_kb(filename):
    res = None
    with open(filename, "rb") as f:
        res = pickle.load(f)
    return res

# Visualize KB

In [69]:
def save_network_html(kb, filename="network.html"):

    color_map = {
        'Governance Negative': '#070245',
        'Governance Neutral': '#4D6CB9',
        'Governance Positive': '#72EFF5',
        'Environmental Negative': '#214124',
        'Environmental Neutral':  '#29A734',
        'Environmental Positive': '#28EA1C',
        'Social Negative': '#CEB501',
        'Social Neutral': '#E6CA00',
        'Social Positive': '#FAFA03'
    }
    # create network
    net = Network(directed=True, width="700px", height="700px", bgcolor="#eeeeee", notebook=True)

    # nodes
    color_entity = "#A699B0"
    for e in kb.entities:
        net.add_node(e, shape="circle", color=color_entity)

    # edges
    for r in kb.relations:
        # print(r)
        if r['head'] not in kb.entities or r['tail'] not in kb.entities:
          continue
        if 'sentiment' in r:
          net.add_edge(r["head"], r["tail"],
                      title=r["type"] + f'({r["sentiment"]})', label=r["type"], color=color_map[r['sentiment']])
        else:
          net.add_edge(r["head"], r["tail"],
                      title=r["type"], label=r["type"], color=color_entity)
        
    # save network
    net.repulsion(
        node_distance=200,
        central_gravity=0.2,
        spring_length=200,
        spring_strength=0.05,
        damping=0.09
    )
    net.set_edge_smooth('dynamic')
    # print(net)
    net.show(filename)

In [63]:
urls = """https://www.reuters.com/article/climate-change-carbon-targets/factbox-big-oils-climate-targets-idUSL8N2HO1B4
https://carbontracker.org/reports/fault-lines/	
https://www.iigcc.org/news/investor-groups-call-on-companies-to-reflect-climate-related-risks-in-financial-reporting/
https://www.unpri.org/sustainability-issues/accounting-for-climate-change
https://www.iigcc.org/download/investor-expectations-for-paris-aligned-accounts/?wpdmdl=4001&masterkey=5fabc4d15595d
https://cdn.ifrs.org/-/media/feature/news/2019/november/in-brief-climate-change-nick-anderson.pdf?la=en
https://www.iea.org/reports/world-energy-outlook-2020/achieving-net-zero-emissions-by-2050	
https://www.climatechangenews.com/2019/06/14/countries-net-zero-climate-goal/"""
urls = urls.split('\n')

In [ ]:
# news_links = get_news_links("Google", pages=5, max_links=20)
# kb = from_urls_to_kb(news_links, verbose=True)
# filename = "network_3_google.html"
# save_network_html(kb, filename=filename)
# save_kb(kb, filename.split(".")[0] + ".p")
# IPython.display.HTML(filename=filename)

In [79]:
kb = from_urls_to_kb(urls, verbose=True)
kb.print()

8 links to visit
Visiting https://www.reuters.com/article/climate-change-carbon-targets/factbox-big-oils-climate-targets-idUSL8N2HO1B4...
Visiting https://carbontracker.org/reports/fault-lines/	...
Visiting https://www.iigcc.org/news/investor-groups-call-on-companies-to-reflect-climate-related-risks-in-financial-reporting/...
Visiting https://www.unpri.org/sustainability-issues/accounting-for-climate-change...
Visiting https://www.iigcc.org/download/investor-expectations-for-paris-aligned-accounts/?wpdmdl=4001&masterkey=5fabc4d15595d...
Visiting https://cdn.ifrs.org/-/media/feature/news/2019/november/in-brief-climate-change-nick-anderson.pdf?la=en...
Visiting https://www.iea.org/reports/world-energy-outlook-2020/achieving-net-zero-emissions-by-2050	...
Visiting https://www.climatechangenews.com/2019/06/14/countries-net-zero-climate-goal/...
Entities:
  ('Equinor', {'url': ''})
  ('Norway', {'url': ''})
  ('Repsol', {'url': ''})
  ('Spain', {'url': ''})
  ("Norway's", {'url': ''})
  ('c

In [80]:
filename = "chev_2021_net_zero_news2.html"

In [81]:
save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

chev_2021_net_zero_news2.html


In [ ]:
kb.relations

[{'head': 'Vancouver',
  'type': 'capital of',
  'tail': 'British Columbia',
  'sentiment': 'Governance Neutral',
  'meta': {'https://finance.yahoo.com/news/equinox-gold-publishes-2022-esg-200000910.html': {'spans': [[0,
      128]]}}},
 {'head': 'British Columbia',
  'type': 'capital',
  'tail': 'Vancouver',
  'sentiment': 'Governance Neutral',
  'meta': {'https://finance.yahoo.com/news/equinox-gold-publishes-2022-esg-200000910.html': {'spans': [[0,
      128]]}}},
 {'head': 'Sustainable Development Goals',
  'type': 'has part',
  'tail': 'Kyoto Protocol',
  'meta': {'https://finance.yahoo.com/news/equinox-gold-publishes-2022-esg-200000910.html': {'spans': [[242,
      370]]}}},
 {'head': 'Wind',
  'type': 'subclass of',
  'tail': 'Renewable energy',
  'sentiment': 'Environmental Positive',
  'meta': {'https://finance.yahoo.com/news/equinox-gold-publishes-2022-esg-200000910.html': {'spans': [[363,
      491]]}}},
 {'head': 'Solar power',
  'type': 'subclass of',
  'tail': 'Renewable e